In [ ]:
# The code was removed by Watson Studio for sharing.

# Import data

In [84]:
# The code was removed by Watson Studio for sharing.

# Imports

In [85]:
from pyspark.sql.functions import when, count, col, isnan, countDistinct,from_unixtime,from_utc_timestamp, unix_timestamp,split
import seaborn as sns 
import matplotlib.pyplot as plt 

# Define variables

In [86]:
# Define data information

# Label
colLabel = ["Severity"]
# Categorical values
colCat = ['TMC',
        'Street',
        'Side',
        'City',
        'County',
        'State',
        'Zipcode',
        'Country',
        'Timezone',
        'Airport_Code',
        'Wind_Direction',
        'Weather_Condition',
        'Amenity',
        'Bump',
        'Crossing',
        'Give_Way',
        'Junction',
        'No_Exit',
        'Railway',
        'Roundabout',
        'Station',
        'Stop',
        'Traffic_Calming',
        'Traffic_Signal',
        'Turning_Loop',
        'Sunrise_Sunset',
        'Civil_Twilight',
        'Nautical_Twilight',
        'Astronomical_Twilight',
]

# Removable columns (Since we dont have the tools for it at the moment, example: Text analysis)
colRem = ['ID', 'Source','End_Time', 'End_Lat', 'End_Lng', 'Description']

# Numerical values
colNum = list(filter(lambda x: x not in colLabel and x not in colCat and x not in colRem , df.columns))

print(f"Categories:\n \
Labels: {len(colLabel)}\n \
Classes: {len(colCat)}\n \
Removed: {len(colRem)}\n \
Numerical: {len(colNum)}") 

print(f"Rows: {df.count()}\nColumns {len(df.columns)}")
df.printSchema()
df.take(5)

Categories:
 Labels: 1
 Classes: 29
 Removed: 6
 Numerical: 13
Rows: 2974335
Columns 49
root
 |-- ID: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- TMC: string (nullable = true)
 |-- Severity: string (nullable = true)
 |-- Start_Time: string (nullable = true)
 |-- End_Time: string (nullable = true)
 |-- Start_Lat: string (nullable = true)
 |-- Start_Lng: string (nullable = true)
 |-- End_Lat: string (nullable = true)
 |-- End_Lng: string (nullable = true)
 |-- Distance(mi): string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Number: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- Side: string (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Airport_Code: string (nullable = true)
 |-- Weather_Timestamp: string (nullable = tru

[Row(ID='A-1', Source='MapQuest', TMC='201.0', Severity='3', Start_Time='2016-02-08 05:46:00', End_Time='2016-02-08 11:00:00', Start_Lat='39.865147', Start_Lng='-84.058723', End_Lat=None, End_Lng=None, Distance(mi)='0.01', Description='Right lane blocked due to accident on I-70 Eastbound at Exit 41 OH-235 State Route 4.', Number=None, Street='I-70 E', Side='R', City='Dayton', County='Montgomery', State='OH', Zipcode='45424', Country='US', Timezone='US/Eastern', Airport_Code='KFFO', Weather_Timestamp='2016-02-08 05:58:00', Temperature(F)='36.9', Wind_Chill(F)=None, Humidity(%)='91.0', Pressure(in)='29.68', Visibility(mi)='10.0', Wind_Direction='Calm', Wind_Speed(mph)=None, Precipitation(in)='0.02', Weather_Condition='Light Rain', Amenity='False', Bump='False', Crossing='False', Give_Way='False', Junction='False', No_Exit='False', Railway='False', Roundabout='False', Station='False', Stop='False', Traffic_Calming='False', Traffic_Signal='False', Turning_Loop='False', Sunrise_Sunset='Nigh

# Get missing data

In [87]:
# Check data

df_missingVals_colNum = df.select([count(when(col(c).isNull(), c)).alias(c) for c in colNum])
df_missingVals_colCat = df.select([count(when(col(c).isNull(), c)).alias(c) for c in colCat])

#df_missingVals_colNum_proc = df.select([(count(when(col(c).isNull(), c))/count(c)).alias(c) for c in colNum])
#df_missingVals_colCat_proc = df.select([(count(when(col(c).isNull(), c))/count(c)).alias(c) for c in colCat])

# Missing value in each column
#df.filter(df_missingVals_colNum.collect().asDict() > )
df_missingVals_colCat.collect()
df_missingVals_colNum.collect()



[Row(Start_Time=0, Start_Lat=0, Start_Lng=0, Distance(mi)=0, Number=1917605, Weather_Timestamp=36705, Temperature(F)=56063, Wind_Chill(F)=1852623, Humidity(%)=59173, Pressure(in)=48142, Visibility(mi)=65691, Wind_Speed(mph)=440840, Precipitation(in)=1998358)]

In [88]:
df_missingVals_colNum.groupBy()

rdd = sc.parallelize(df_missingVals_colNum.collect())
rdd.filter(lambda c: int(c) > 10000).collect()
# TODO: NOT WORKING


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 166.0 failed 4 times, most recent failure: Lost task 1.3 in stage 166.0 (TID 1380, 172.30.72.23, executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/ibm/spark/python/lib/pyspark.zip/pyspark/worker.py", line 253, in main
    process()
  File "/opt/ibm/spark/python/lib/pyspark.zip/pyspark/worker.py", line 248, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/ibm/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 379, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/ibm/spark/python/lib/pyspark.zip/pyspark/util.py", line 55, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-88-0d805c00fa4f>", line 4, in <lambda>
TypeError: int() argument must be a string, a byte

In [ ]:
# How many that is above 30%
missingLimit = df.count() - df.count()*0.3 # Must be more than this amound: 1.0 - 1.0*0.3 = 0.7 of total set

#df.select([when(count(when(col(c).isNull(), c)) > missingLimit, c).alias(c) for c in df.columns]).collect()[0].asDict()
#df.filter(lambda c: print(type(c)))
#df.where([when(count(when(col(c).isNull(), c)) > missingLimit, c).alias(c) for c in df.columns] != None).show()

df.agg(*[
    when(count(c) > missingLimit,c).alias(c)    # vertical (column-wise) operations in SQL ignore NULLs
    for c in df.columns
]).show()

In [ ]:
rdd_data.collect()

# Get categorical values

In [ ]:
# Checking how many classes that can be used
df.agg(*(countDistinct(col(c)).alias(c) for c in colCat)).collect()[0].asDict()

# Causes

## Frequency of severity and state

In [ ]:
state_freq = df.groupBy('State').count().orderBy('count',ascending=False).collect()

In [ ]:
rdd = sc.parallelize(state_freq)
pd_states = rdd.toDF().toPandas()

# Plot data
fig,ax = plt.subplots(figsize=(16,10))
sns.barplot(pd_states['State'],pd_states['count'])

In [ ]:
state_severity_freq = df.groupBy('State','Severity').count().orderBy('count',ascending=False).collect()


In [ ]:
state_severity_freq

In [ ]:
rdd = sc.parallelize(state_severity_freq)
hurr = rdd.filter(lambda c: rdd['Severity'] == '1')

pd_states = rdd.toDF().toPandas()


# 
# Plot data
fig,ax = plt.subplots(figsize=(16,10))
#sns.barplot(pd_states['State']=='1',pd_states['count'])
#sns.barplot(pd_states['State']=='2',pd_states['count'])
#sns.barplot(pd_states['State']=='3',pd_states['count'])
#sns.barplot(pd_states['State']=='4',pd_states['count'])

## Weather conditions

In [ ]:
weather_freq = df.groupBy('Weather_Condition').count().orderBy('count',ascending=False).collect()

In [ ]:
#display_limit = 12 # Limit on how many objects that should be displayed

fig, ax=plt.subplots(figsize=(16,25))

rdd = sc.parallelize(weather_freq)
pd_weather = rdd.toDF().toPandas()

b = sns.barplot(pd_weather['count'][:],pd_weather['Weather_Condition'][:], color="blue")

b.axes.set_title("Weather Condition for accidents",fontsize=20)
b.set_xlabel("Number of Accidents",fontsize=15)
b.set_ylabel("Weather_Condition",fontsize=15)
b.tick_params(labelsize=10)
plt.show()


In [ ]:
# How severe was the condition for each type of weather?

## Time when accidents occured

## Development of accidents on year

# Preprocessing

In [ ]:
# Drop nan - Dont do this until cleaning above is fixed



# Drop features that is less than 0.01% chance
# Impute dataset
# Standarlize
# One-hot encoding

In [ ]:


# Todo numeric value

## One-hot encoding

# Feature importance

# Pipeline